In [2]:
import pandas as pd
import numpy as np
import sklearn
import ast

In [3]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [4]:
credits.rename(columns={'movie_id':'id'},inplace=True)
movies = movies[['id','keywords','overview','title','genres']]
new_df= movies.merge(credits,on='id',).drop('title_y',axis=1).rename(columns={'title_x':'title'})

In [5]:
new_df.head(1)

,id,keywords,overview,title,genres,cast,crew
0,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
new_df.head(1)['genres'].values

array(['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'],
      dtype=object)

In [7]:
def converter(obj):
    list=[]
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list

In [8]:
new_df['genres'] = new_df['genres'].apply(converter)

In [9]:
new_df['cast']= new_df['cast'].apply(converter)

In [10]:
def convertcrew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
    return l

In [11]:
new_df['crew']=new_df['crew'].apply(convertcrew)

In [12]:
new_df['keywords']=new_df['keywords'].apply(converter)

In [13]:
new_df.dropna(inplace=True)

In [14]:
new_df['overview']= new_df['overview'].apply(lambda x: x.split())

In [15]:
new_df['tags']=new_df['overview']+new_df['keywords']+new_df['genres']+new_df['cast']+new_df['crew']

In [16]:
new_df['tags']=new_df['tags'].apply(lambda x: [i.replace(' ','') for i in x])

In [17]:
new_df= new_df.iloc[:,[0,3,7]]

In [18]:
new_df['tags']=new_df['tags'].apply(lambda x: ' '.join(x))

In [19]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cvr=CountVectorizer(max_features=5000,stop_words='english')
vectors=cvr.fit_transform(new_df['tags']).toarray()

In [21]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(t):
    stlist=[]
    for i in t.split():
        stlist.append(ps.stem(i))
    return ' '.join(stlist)

In [22]:
new_df['tags']=new_df['tags'].apply(stem)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [24]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])

[(0, 1.0000000000000007),
 (1914, 0.23473823893078544),
 (1213, 0.23294541397390256),
 (582, 0.23097828906119441),
 (539, 0.2252817784447915),
 (507, 0.2191252450446388),
 (1438, 0.21398024625545647),
 (3603, 0.21398024625545647),
 (4185, 0.2123976976214366),
 (260, 0.21170244960998522),
 (74, 0.20935894733965596),
 (83, 0.20256711147285467),
 (2966, 0.19839002137983244),
 (3723, 0.19738550848793068),
 (495, 0.19304683562633607),
 (2327, 0.19258222162991084),
 (322, 0.1904761904761905),
 (61, 0.1889822365046136),
 (1201, 0.18824832213400472),
 (3322, 0.18712029714127995),
 (36, 0.18609684207969418),
 (2403, 0.18609684207969418),
 (1191, 0.18523964340873708),
 (1531, 0.18523964340873708),
 (2198, 0.18257418583505536),
 (3533, 0.18257418583505536),
 (972, 0.18257418583505533),
 (2780, 0.18156825980064073),
 (4399, 0.17817416127494962),
 (1318, 0.17699808135119716),
 (47, 0.17699808135119713),
 (4041, 0.17636890537566463),
 (1326, 0.17496355305594127),
 (942, 0.17295817388759027),
 (151, 

In [25]:
def recommend(movie):
    movie_index= new_df[new_df['title']== movie].index[0]
    distances = similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    '''mvindex = [movie_list[i][0] for i in range(0,5)]
    mvmovie = [new_df.iloc[i].title for i in mvindex]
    print(mvmovie)'''
    print(movie_list)
recommend('Batman Begins')

[(65, 0.3954824982302081), (3, 0.3355608839748466), (4635, 0.3142696805273545), (210, 0.2797154638927579), (1359, 0.2732971972499743)]


In [26]:
new_df.iloc[129].title

'Thor'

In [29]:
import pickle

In [30]:
pickle.dump(new_df,open('moviesdata.pkl','wb'))

In [31]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [32]:
new_df

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4798,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4799,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4800,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4801,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...
